<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/BX-gpt-app/gemma_ko_2b_rag_chunking_CBP_certi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 라이브러리 설치

In [1]:
# !pip install -q datasets pandas pymongo sentence_transformers
# !pip install -q -U transformers
# !pip install -q accelerate
# !pip install -q "pymongo[srv]"
# !pip install -q -U bitsandbytes
# !pip install -q PyPDF2 openparse
!pip install -q langchain langchain_experimental langchain_openai
# !pip install -q unstructured > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

# 2. Dataset

In [9]:
test_doc_md = '/content/drive/MyDrive/AI-modeling/BX-gpt-app/22-BX-CBP-상품컴포넌트.md'

## 마크다운 문서를 binary 모드로 읽어 디코딩

In [2]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

# 파일을 바이너리 모드로 읽고 디코딩
def read_file_with_encoding(file_path, encodings=['utf-8', 'latin-1', 'cp1252']):
    for encoding in encodings:
        try:
            with open(file_path, 'rb') as file:
                binary_data = file.read()
                return binary_data.decode(encoding)
        except UnicodeDecodeError as e:
            print(f"Failed to decode with encoding {encoding}: {e}")
    raise ValueError("Unable to decode the file with provided encodings.")

# 파일 읽기
# markdown_text = read_file_with_encoding(test_doc_md)

# data = [Document(page_content=markdown_text)]
# print(data[0])
# readme_content = data[0].page_content
# print(readme_content[:10000])

## 개행/svg 파일 제거

In [3]:
import re

def remove_empty_lines(text):
    # 여러 개의 개행 문자를 하나의 개행 문자로 대체
    text = re.sub(r'\n\s*\n', '\n', text)
    return text

def remove_image_lines(text):
    # ![...](...svg) 또는 ![...](...png)로 끝나는 라인
    pattern_svg_jpg = r'^.*!\[.*?\]\(.*?\.(svg|png)\).*$'
    # single hash
    pattern_single_hash = r'^# .*$'
    cleaned_text = re.sub(pattern_svg_jpg, '', text, flags=re.MULTILINE)

    lines = cleaned_text.split('\n')
    cleaned_lines = [line for line in lines if not re.match(pattern_single_hash, line)]
    cleaned_text = '\n'.join(cleaned_lines)
    return cleaned_text

# cleaned_text = remove_empty_lines(data[0].page_content)
# cleaned_text = remove_image_lines(cleaned_text)
# cleaned_text = remove_empty_lines(cleaned_text)
# print(cleaned_text)

## double hash 기준 chunking/반복된 hash 제거

In [4]:
def chunk_by_double_hash(text):
    # ##을 기준으로 문서를 청킹
    pattern = r'(?<=\n)## .*(?=\n)'
    # 패턴을 기준으로 분할
    chunks = re.split(pattern, text)
    # 첫 번째 청크는 빈 문자열이 될 수 있으므로 제거
    if chunks[0].strip() == '':
        chunks = chunks[1:]
    return chunks

def remove_repeated_triple_hash(chunks):
    cleaned_chunks = []
    for chunk in chunks:
        lines = chunk.split('\n')
        cleaned_lines = []
        triple_hash_count = 0
        for line in lines:
            if line.startswith('###'):
                triple_hash_count += 1
                if triple_hash_count > 1:
                    continue  # 연속된 ### 라인이 5줄 이하인 경우 제거
            else:
                triple_hash_count = 0
            cleaned_lines.append(line)

        cleaned_chunks.append('\n'.join(cleaned_lines))
    return cleaned_chunks

# chunks = chunk_by_double_hash(cleaned_text)
# ###이 5줄 이내로 반복되는 라인 제거
# final_chunks = remove_repeated_triple_hash(chunks)

# 결과 출력
# for i, chunk in enumerate(final_chunks):
#     print(f"Chunk {i + 1}:")
#     print(chunk.strip())  # 앞뒤 공백 제거
#     print("===" * 20)

In [5]:
pdf_files = [
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/01-BX-CBP-제품소개.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/02-BX-PF-제품소개.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/02-BX-PF-제품소개.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/12-BX-CBP-공통.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/21-BX-CBP-베이스컴포넌트.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/22-BX-CBP-상품컴포넌트.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/23-BX-CBP-액터컴포넌트.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/24-BX-CBP-계약컴포넌트.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/25-BX-CBP-정산컴포넌트.md',
]

texts = []

for pdf_file in pdf_files:
    print(f"Start Processing {pdf_file}...")

    curr_markdown_text = read_file_with_encoding(pdf_file)

    curr_data = [Document(page_content=curr_markdown_text)]
    curr_cleaned_text = remove_empty_lines(curr_data[0].page_content)
    curr_cleaned_text = remove_image_lines(curr_cleaned_text)
    curr_cleaned_text = remove_empty_lines(curr_cleaned_text)
    curr_chunks = chunk_by_double_hash(curr_cleaned_text)

    final_chunks = remove_repeated_triple_hash(curr_chunks)

    for i, chunk in enumerate(final_chunks):

      # print(f"Chunk {i + 1}:")
      # print(chunk.strip())
      # print("===" * 30)

      texts.append(chunk.strip())

    print(f"End Processing {pdf_file}...")

print(texts)

for i, text in enumerate(texts):
    print(text)
    print("===" * 30)

Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/01-BX-CBP-제품소개.md...
End Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/01-BX-CBP-제품소개.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/02-BX-PF-제품소개.md...
End Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/02-BX-PF-제품소개.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/02-BX-PF-제품소개.md...
End Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/02-BX-PF-제품소개.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/12-BX-CBP-공통.md...
End Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/12-BX-CBP-공통.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/21-BX-CBP-베이스컴포넌트.md...
End Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/21-BX-CBP-베이스컴포넌트.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/22-BX-CBP-상품컴포넌트.md...
End Processing /cont